# OptNet/qpth Example Sudoku Notebook

*By [Brandon Amos](https://bamos.github.io) and [J. Zico Kolter](http://zicokolter.com/).*

---

This notebook is released along with our paper
[OptNet: Differentiable Optimization as a Layer in Neural Networks](https://arxiv.org/abs/1703.00443).

This notebook shows an example of constructing an
OptNet layer in PyTorch with our [qpth library](https://github.com/locuslab/qpth)
to solve [the game Sudoku](https://en.wikipedia.org/wiki/Sudoku)
as a prediction problem from data.
See [our qpth documentation page](https://locuslab.github.io/qpth/)
for more details on how to use `qpth`.
The experiments for our paper that use this library are in
[this repo](https://github.com/locuslab/optnet).
Specifically [here](https://github.com/locuslab/optnet/tree/master/sudoku)
is the full source code for the publihsed version of Sudoku.


## Setup and Dependencies

+ Python/numpy/[PyTorch](https://pytorch.org)
+ [qpth](https://github.com/locuslab/qpth):
  *Our fast QP solver for PyTorch released in conjunction with this paper.*
+ [bamos/block](https://github.com/bamos/block):
  *Our intelligent block matrix library for numpy, PyTorch, and beyond.*
+ Optional: [bamos/setGPU](https://github.com/bamos/setGPU):
  A small library to set `CUDA_VISIBLE_DEVICES` on multi-GPU systems.

In [1]:
import os

import sys

import numpy as np
import torch

import torch.nn as nn
from torch.autograd import Function, Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F

from qpth.qp import QPFunction

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

# Setup: Download the data and pretrained model

+ The pre-trained model is for later.
  The following command should download everything to a tmp directory for you
  if you have the `wget` and `tar` commands installed.
+ (Sorry for the bad form here)

In [2]:
tmpDir = "/tmp/optnet.sudoku"
cmd = ('mkdir {}; cd {} &&'
       'wget "http://joule.isr.cs.cmu.edu:11235/optnet/arxiv.v1.sudoku.tgz" && '
       'tar xf arxiv.v1.sudoku.tgz').format(*[tmpDir]*2)
dataDir = os.path.join(tmpDir, 'arxiv.v1.sudoku')
assert os.system(cmd) == 0

sys.path.append(tmpDir+'/arxiv.v1.sudoku')
import models # From /tmp/optnet.sudoku/arxiv.v1.sudoku/models.py

In [3]:
testPct = 0.1

with open('{}/2/features.pt'.format(dataDir), 'rb') as f:
    X = torch.load(f)
with open('{}/2/labels.pt'.format(dataDir), 'rb') as f:
    Y = torch.load(f)

N, nFeatures = X.size(0), int(np.prod(X.size()[1:]))

nTrain = int(N*(1.-testPct))
nTest = N-nTrain

trainX = X[:nTrain]
trainY = Y[:nTrain]
testX = X[nTrain:]
testY = Y[nTrain:]

## What the data for the Sudoku task looks like

The inputs are incomplete boards and the outputs
are the completed boards. Here's what the first
input and output in the test set looks like.

In [4]:
def decode_onehot(encoded_board):
    """Take the unique argmax of the one-hot encoded board."""
    v,I = torch.max(encoded_board, 0)
    return ((v>0).long()*(I+1)).squeeze()

print("First testing example input (unsolved Sudoku board): ", decode_onehot(testX[0]))
print("First testing example output (solved Sudoku board): ", decode_onehot(testY[0]))

First testing example input (unsolved Sudoku board):  
 0  0  2  4
 3  0  0  1
 0  4  0  0
 0  0  3  0
[torch.LongTensor of size 4x4]

First testing example output (solved Sudoku board):  
 1  3  2  4
 3  2  4  1
 2  4  1  3
 4  1  3  2
[torch.LongTensor of size 4x4]



You may have noticed that we had to decode those examples.
That's because they're actually *one-hot encoded* for how
we're going to model the task.
That means that instead of representing the values as 
something between 1 and 4, they're represented
as a 4-dimensional vector with a 1 in the index of the value.
Here's what the same first example from the test set
actually looks like:

In [5]:
print("First test example input one-hot encoded (unsolved Sudoku board): ", testX[0])
print("First test example output one-hot encoded (solved Sudoku board): ", testY[0])

First test example input one-hot encoded (unsolved Sudoku board):  
(0 ,.,.) = 
  0  0  0  0
  0  0  0  1
  0  0  0  0
  0  0  0  0

(1 ,.,.) = 
  0  0  1  0
  0  0  0  0
  0  0  0  0
  0  0  0  0

(2 ,.,.) = 
  0  0  0  0
  1  0  0  0
  0  0  0  0
  0  0  1  0

(3 ,.,.) = 
  0  0  0  1
  0  0  0  0
  0  1  0  0
  0  0  0  0
[torch.FloatTensor of size 4x4x4]

First test example output one-hot encoded (solved Sudoku board):  
(0 ,.,.) = 
  1  0  0  0
  0  0  0  1
  0  0  1  0
  0  1  0  0

(1 ,.,.) = 
  0  0  1  0
  0  1  0  0
  1  0  0  0
  0  0  0  1

(2 ,.,.) = 
  0  1  0  0
  1  0  0  0
  0  0  0  1
  0  0  1  0

(3 ,.,.) = 
  0  0  0  1
  0  0  1  0
  0  1  0  0
  1  0  0  0
[torch.FloatTensor of size 4x4x4]



# Defining a model for this task

We've now turned (mini-)Sudoku into a machine learning task that
you can apply any model and learning algorithm to. 
In this notebook, we'll just show how to initialize and train
an OptNet model for this task.
However you can play around and swap this out for any
model you want!
Check out [our baseline models](https://github.com/locuslab/optnet/blob/master/sudoku/models.py)
if you're interested.

Sudoku is actually an integer programming problem but
we can relax it to an LP (or LP with a small ridge term,
which we'll actually use) that can be expressed as:

```
y* = argmin_y 0.5 eps y^T y - p^T y
         s.t. Ay  = b
               y >= 0
```

To quickly explain this, the quadratic term `0.5 eps y^T y`
is a small ridge term so we can use `qpth`,
`p` is the (flattened) one-hot encoded input,
the `-p^T y` term constrains the solution to contain
the same pieces as the unsolved board,
and the linear equality constraints `Ay = b`
encode the constraints of Sudoku (the row, columns,
and sub-blocks must contain all of the digits).

If you want to check your understanding of this:

1. What do some example constraints `a_i^T y = b_i` look like?
2. What happens if we remove the linear equality constraint?

Implementing this model is just a few lines of PyTorch with our qpth library.
Note that in this notebook we'll just execute this on the CPU,
but for performance reasons you should use a GPU for serious
experiments:

In [6]:
class OptNet(nn.Module):
    def __init__(self, n, Qpenalty):
        super().__init__()
        nx = (n**2)**3
        self.Q = Variable(Qpenalty*torch.eye(nx).double())
        self.G = Variable(-torch.eye(nx).double())
        self.h = Variable(torch.zeros(nx).double())
        A_shape = (40, 64) # Somewhat magic, it's from the true solution.
        self.A = Parameter(torch.rand(A_shape).double())
        self.b = Variable(torch.ones(A_shape[0]).double())

    def forward(self, puzzles):
        nBatch = puzzles.size(0)

        p = -puzzles.view(nBatch, -1)

        return QPFunction(verbose=-1)(
            self.Q, p.double(), self.G, self.h, self.A, self.b
        ).float().view_as(puzzles)

That's it! Let's randomly initialize this model and see what it does on the first test set example. What do you expect?

In [7]:
model = OptNet(2, 0.1)
pred = model(Variable(testX[0].unsqueeze(0))).squeeze().data

print("First test example input (unsolved Sudoku board): ", decode_onehot(testX[0]))
print("First test example output (TRUE solved Sudoku board): ", decode_onehot(testY[0]))
print("First test example prediction: ", decode_onehot(pred))

First test example input (unsolved Sudoku board):  
 0  0  2  4
 3  0  0  1
 0  4  0  0
 0  0  3  0
[torch.LongTensor of size 4x4]

First test example output (TRUE solved Sudoku board):  
 1  3  2  4
 3  2  4  1
 2  4  1  3
 4  1  3  2
[torch.LongTensor of size 4x4]

First test example prediction:  
 3  4  1  2
 2  3  3  2
 3  1  2  3
 1  1  2  2
[torch.LongTensor of size 4x4]



Wow that prediction is way off!! That's expected since the model was randomly initialized. Note that at this point, some of the constraints actually make it impossible to match the unsolved board (like the `4` at the top right corner)

Let's look a random nonsense constraint that the model just satisfied. Here are the coefficients in the first row, `a_1` and `b_1`. The last line here
shows that the constraint is acutally satisfied (up to machine precision).

In [8]:
np.set_printoptions(precision=2)
a0 = model.A[0].data.numpy()
b0 = model.b.data[0]
z = pred.numpy().ravel()
print('First row of A:\n', a0)
print('-'*30)
print('First entry of b: ', b0)
print('-'*30)
print('a0^T z - b: ', np.dot(a0, z) - b0)

First row of A:
 [ 0.49  0.74  0.66  0.42  0.95  0.73  0.83  0.2   0.39  0.63  0.36  0.25
  0.4   0.26  0.81  0.88  0.98  0.61  0.89  0.9   0.51  0.15  0.11  0.82
  0.14  0.22  0.56  0.13  0.64  0.82  0.92  0.25  0.2   0.04  0.25  0.63
  0.07  0.68  0.78  0.34  0.62  0.01  0.72  0.54  0.07  0.41  0.43  0.18
  0.02  0.21  0.62  0.81  0.3   0.97  0.29  0.51  0.87  0.43  0.6   0.14
  0.15  0.16  0.15  0.69]
------------------------------
First entry of b:  1.0
------------------------------
a0^T z - b:  -5.92086668583e-09


# Training the model

Let's start training this model my comparing the predictions
to the true solutions and taking gradient steps.
This takes a while to run (overnight on a GPU), so here
we'll just take 10 steps through the first 10 training examples
to illustrate what the full training would look like.

In [9]:
loss_fn = torch.nn.MSELoss()

# Initialize the optimizer.
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(10):
    x_batch = Variable(trainX[t].unsqueeze(0))
    y_batch = Variable(trainY[t].unsqueeze(0))
    
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x_batch)

    # Compute and print loss.
    loss = loss_fn(y_pred, y_batch)
    print('Iteration {}, loss = {:.2f}'.format(t, loss.data[0]))

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

Iteration 0, loss = 0.24
Iteration 1, loss = 0.24
Iteration 2, loss = 0.24
Iteration 3, loss = 0.24
Iteration 4, loss = 0.24
Iteration 5, loss = 0.23
Iteration 6, loss = 0.24
Iteration 7, loss = 0.24
Iteration 8, loss = 0.22
Iteration 9, loss = 0.24


# Looking at a pre-trained model

Imagine you kept that running for a while.
Let's load my pre-trained model we downloaded earlier and
see the predictions on the first test example again:

In [10]:
A_file = os.path.join(tmpDir, 'arxiv.v1.sudoku', 'pretrained-optnet-A.pth')
trainedA = torch.load(A_file)

trainedModel = OptNet(2, 0.2)
trainedModel.A.data = trainedA

pred = trainedModel(Variable(testX[0].unsqueeze(0))).data.squeeze()

print("First test example input (unsolved Sudoku board): ", decode_onehot(testX[0]))
print("First test example output (TRUE solved Sudoku board): ", decode_onehot(testY[0]))
print("First test example prediction: ", decode_onehot(pred))

First test example input (unsolved Sudoku board):  
 0  0  2  4
 3  0  0  1
 0  4  0  0
 0  0  3  0
[torch.LongTensor of size 4x4]

First test example output (TRUE solved Sudoku board):  
 1  3  2  4
 3  2  4  1
 2  4  1  3
 4  1  3  2
[torch.LongTensor of size 4x4]

First test example prediction:  
 1  3  2  4
 3  2  4  1
 2  4  1  3
 4  1  3  2
[torch.LongTensor of size 4x4]



We did it! With just a few lines of code we've trained
an intuitive model that solves Sudoku.

As a closing note, what does the trained `A` matrix look like?
With this formulation, we don't expect it to be the nice,
sparse coefficient matrix encoding the rules we typically
think of Sudoku as since any row-transformed
version of this matrix is an equivalent valid solution:

In [11]:
trainedA


 1.9467  0.9184 -0.3419  ...  -3.1399 -0.6383  1.9654
 1.2744 -3.1572  0.0089  ...   5.6140  0.4215 -1.9954
 2.1541  0.9581  0.6416  ...   1.2639 -1.1828  3.2781
          ...             ⋱             ...          
-2.5137  1.8118  0.8839  ...   1.8048  0.6488  1.2468
-1.2608 -1.9652  1.1355  ...   3.6201  0.9974  9.0308
-4.3232 -0.6153  0.8325  ...   1.4742  2.2705 -1.1042
[torch.DoubleTensor of size 40x64]